In [1]:
import re
import unicodedata
from pathlib import Path
import pandas as pd



In [2]:
INPUT_PATH = Path("/home/onizuka/Bureau/ai_project/dataset/whatsapp_syndic_texts_100.csv")   
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True, parents=True)
pd.set_option("display.max_colwidth", 120)



In [3]:
df = pd.read_csv(INPUT_PATH, dtype=str)

print("Raw shape:", df.shape)
print("Columns:", df.columns.tolist())


Raw shape: (100, 4)
Columns: ['message_id', 'datetime', 'residence_id', 'text']


## CHECK SCHEMA OF EXCEL FILE

In [4]:
expected = {"message_id", "datetime", "residence_id", "text"}
missing = expected - set(df.columns)
if missing:
    raise ValueError(f"Colonnes manquantes: {missing}. Colonnes trouvées: {df.columns.tolist()}")


## NORMALISATION

In [5]:
def normalize_whitespace(s: str) -> str:
    # Convert to string safely
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s = str(s)
    # Normalize unicode (keeps accents, standardizes forms)
    s = unicodedata.normalize("NFKC", s)
    # Replace common weird spaces
    s = s.replace("\u00A0", " ").replace("\u200B", " ")
    # Trim + collapse spaces
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s


In [6]:
# normalize core columns
df["message_id"] = df["message_id"].apply(normalize_whitespace)
df["datetime"] = df["datetime"].apply(normalize_whitespace)
df["residence_id"] = df["residence_id"].apply(normalize_whitespace)
df["text"] = df["text"].apply(normalize_whitespace)



## Handle null values

In [7]:
# drop rows where message_id or text is empty
before = len(df)
df = df[(df["message_id"] != "") & (df["text"] != "")]
df = df.drop_duplicates(subset=["message_id"], keep="first")
after = len(df)

print(f"Dropped {before - after} empty/duplicate rows. New shape: {df.shape}")


Dropped 0 empty/duplicate rows. New shape: (100, 4)


## remove emojies


In [8]:
def remove_emojis(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Unicode ranges for emojis & symbols
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map
        "\U0001F700-\U0001F77F"
        "\U0001F780-\U0001F7FF"
        "\U0001F800-\U0001F8FF"
        "\U0001F900-\U0001F9FF"
        "\U0001FA00-\U0001FAFF"
        "\U00002700-\U000027BF"  # dingbats
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(" ", text)


##  Create text_clean (safe cleaning, NOT destructive)


In [9]:
"""
# Objectif: nettoyer sans perdre accents / sens.
# - lower
# - uniformize punctuation spacing
# - keep Arabic letters, keep digits, keep most punctuation
# - collapse spaces

"""


def make_text_clean(s: str) -> str:
    s = normalize_whitespace(s)
    s = s.lower()

    # remove emojis BEFORE anything else
    s = remove_emojis(s)

    # normalize quotes
    s = s.replace("’", "'").replace("“", '"').replace("”", '"')

    # keep ? ! . , : ; (important for intent)
    s = re.sub(r"([,;:()])", r" \1 ", s)

    # remove duplicated punctuation (!!! -> !!)
    s = re.sub(r"([!?])\1{2,}", r"\1\1", s)

    # collapse spaces
    s = re.sub(r"\s+", " ", s).strip()
    return s



In [10]:
df["text_clean"] = df["text"].apply(make_text_clean)


In [11]:
def clean_punct_for_ml(s: str) -> str:
    # replace these by spaces, keep . ? !
    s = re.sub(r"[:,;()\[\]{}\/\\]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_clean"] = df["text_clean"].apply(clean_punct_for_ml)


In [12]:
# Ajouter la colonne channel si elle n'existe pas
if "channel" not in df.columns:
    df["channel"] = "whatsapp"


In [13]:
# Ajouter la colonne has_media si elle n'existe pas
if "has_media" not in df.columns:
    df["has_media"] = 0


In [14]:
df

,message_id,datetime,residence_id,text,text_clean,channel,has_media
0,MSG_SI9NB74X2EK3,2025-12-27 01:42:18,RES_RABAT_RIHAB,Ascenseur en panne depuis ce matin près du tableau électrique. Personnes âgées au 5e...,ascenseur en panne depuis ce matin près du tableau électrique. personnes âgées au 5e...,whatsapp,0
1,MSG_1YV2STN7E36H,2026-01-15 09:15:15,RES_FES_ANDALOUS,Ascenseur en panne depuis depuis 3 jours près du tableau électrique. Personnes âgées au 5e...,ascenseur en panne depuis depuis 3 jours près du tableau électrique. personnes âgées au 5e...,whatsapp,0
2,MSG_J88QEOZ0KZ4Z,2026-01-16 04:58:09,RES_RABAT_RIHAB,Pouvez-vous m’envoyer le détail des charges du mois dernier ?,pouvez-vous m'envoyer le détail des charges du mois dernier ?,whatsapp,0
3,MSG_6RIDLOFAS1EK,2026-01-15 07:26:07,RES_FES_ATLAS,C’est quoi la procédure pour réserver la salle polyvalente ?,c'est quoi la procédure pour réserver la salle polyvalente ?,whatsapp,0
4,MSG_FFQOT8HLJEFY,2026-01-18 14:53:54,RES_FES_ATLAS,Réclamation propreté : cage d’escalier A sale depuis depuis 30 min avec des étincelles. 🙏,réclamation propreté cage d'escalier a sale depuis depuis 30 min avec des étincelles.,whatsapp,0
...,...,...,...,...,...,...,...
95,MSG_H55HEWLGJE2A,2025-12-22 08:55:32,RES_CASA_OASIS,À quelle heure passe l’agent de nettoyage normalement ?,à quelle heure passe l'agent de nettoyage normalement ?,whatsapp,0
96,MSG_BI4O6Q0USWOV,2025-12-23 08:33:53,RES_CASA_MARINA,Réclamation propreté : toit sale depuis depuis 1h on entend un bruit bizarre. 😟,réclamation propreté toit sale depuis depuis 1h on entend un bruit bizarre.,whatsapp,0
97,MSG_XL9967I6YUJ7,2026-01-02 22:39:29,RES_CASA_OASIS,Coupure d’électricité dans les parties communes et ça sent fort. Escaliers dans le noir. merci,coupure d'électricité dans les parties communes et ça sent fort. escaliers dans le noir. merci,whatsapp,0
98,MSG_TRFLQBMAE38S,2026-01-13 08:40:54,RES_CASA_MARINA,Le badge / télécommande ne marche plus depuis ce matin. Comment faire ? 😟,le badge télécommande ne marche plus depuis ce matin. comment faire ?,whatsapp,0


In [15]:
df.columns

Index(['message_id', 'datetime', 'residence_id', 'text', 'text_clean',
       'channel', 'has_media'],
      dtype='object')

## Export data

In [16]:
# Export cleaned dataset
OUTPUT_PATH = "../cleanData/messages_processed.csv"

df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")
print(f"✅ Dataset nettoyé sauvegardé dans {OUTPUT_PATH}")


✅ Dataset nettoyé sauvegardé dans ../cleanData/messages_processed.csv
